In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import csv
import time
import random
import re

#setup our webdriver with a chrome profile and navigate to the docket portal
driver = webdriver.Chrome("C:/Users/KNguyen/chromedriver.exe")
driver.get("http://starw1.ncuc.net/NCUC/portal/ncuc/page/DocumentsParameterSearch/portal.aspx")
#SEARCH PAGE
#click checkbox to search by docket type
element2 = driver.find_element_by_css_selector("#ctl00_ContentPlaceHolder1_PortalPageControl1_ctl36_PSCDocumentSearchControl1_filterByFilingTypesCheckBox")
element2.click()                

time.sleep(5)
#select "SP - Application"

element3 = driver.find_element_by_css_selector("#ctl00_ContentPlaceHolder1_PortalPageControl1_ctl36_PSCDocumentSearchControl1_filingTypesList > option:nth-child(790)")
element3.click()

element = driver.find_element_by_css_selector("#ctl00_ContentPlaceHolder1_PortalPageControl1_ctl36_PSCDocumentSearchControl1_searchButton")
element.click()

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

In [18]:
sum_blob = driver.find_element_by_css_selector("#ctl00_ContentPlaceHolder1_PortalPageControl1_ctl36_PSCDocumentSearchControl1_itemCountLabel")
sum_clean_blob = sum_blob.text.split()
print (" ".join(sum_clean_blob))

num_items = int(sum_clean_blob[2])
num_pages = int
if (num_items%10) == 0:
    num_pages = num_items/10
else:
    num_pages = num_items//10 + 1
print ("Number of pages: " + str(num_pages))
next_page = 2

Items Count: 2344
Number of pages: 235


In [19]:
with open('C:/Users/KNguyen/NCUC_master_list_test.csv', 'w', newline='') as ncuc:
    a = csv.writer(ncuc, delimiter=',')
    bill_rows = [['Title', 'Type', 'ID', 'Detail','ID_two','Detail_two', 'Date Filed', 'Link to Document']]
    for x in range(1,num_pages+1):
        time.sleep(2)

        results_dirty = driver.find_element_by_css_selector("#ctl00_ContentPlaceHolder1_PortalPageControl1_ctl36_PSCDocumentSearchControl1_resultsGrid")
        results_html = results_dirty.get_attribute("innerHTML")
        results_segs = results_html.split("<tbody>")
        
        results_segs_sorted = []
        for item in results_segs:
            if ('Filed In' in item):
                results_segs_sorted.append(item)

        for seg in results_segs_sorted:
            bill = seg.split("<td")
            lion = []
            
            bill_name = ''
            doc_type = ''
            doc_id = ''
            for bit in bill:
                format_bit = bit.replace('>','<').split('<')
                lion.append(format_bit)
            bill_name = lion[1][1]

            doc_type = lion[2][1]

            doc_id = lion[5][5]

            link_detail = ""
            doc_id2 = ""
            link_detail2 = ""
            if (len(lion[5])==28):
                dirty_detail = lion[5][4]
                clean_detail = dirty_detail.split('"')
                link_detail = clean_detail[3]
                doc_id2 = lion[5][9]

                dirty_detail2 = lion[5][8]
                clean_detail2 = dirty_detail2.split('"')
                link_detail2 = clean_detail2[3]

            elif (len(lion[5])==20):
                link_detail = 'none'

            else:
                dirty_detail = lion[5][4]
                clean_detail = dirty_detail.split('"')
                link_detail = clean_detail[3]


            #date_filed = lion[7][3]

            link_doc = ""
            match = re.search(r'http://starw1.ncuc.net/NCUC/portal/ncuc/PSC/PSCDocumentDetailsPageNCUC[\w&?=.;-]*', seg)
            if match:
                link_doc = match.group().replace(';','&')
            else:
                link_doc = 'none'
            
            date_filed = lion[7][3]

            newrow = [bill_name, doc_type, doc_id, link_detail, doc_id2, link_detail2, date_filed, link_doc]
            bill_rows.append(newrow)    

            
        header = driver.find_element_by_css_selector(".SearchResultsFooter.text-right")
        ahref = header.find_elements_by_css_selector("a")
        for link in ahref:
            if ((next_page-1)%10 == 0):
                ahref[-1].click()
                driver.implicitly_wait(1)
                next_page += 1
                break
            elif (link.text == str(next_page)):
                next_page += 1
                link.click()
                driver.implicitly_wait(1)
                break

    a.writerows(bill_rows)